## Read Data

In [1]:
import pandas as pd
df= pd.read_csv(r"D:\Downloads\MIA\Task3\f-1-racer-diet-planning\train.csv")
df.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


## Start EDA finding no need for cleaining cuz there is no nulls nor duplictaes 

In [2]:
# from ydata_profiling import ProfileReport
# profile=ProfileReport(df, title = "EDA Report", explorative=True)

# profile.to_file("eda_reportC2.html")

## Importing libraries i`m gonna  need

In [3]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso,ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np



## select feature and target

In [4]:
x = df.drop(columns=['id', 'Calories'])
y = df['Calories']


## encode categorial to numeric 

In [5]:
x['Sex'] = np.where(x['Sex'] == 'male', 1, 0)
x.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,1,36,189.0,82.0,26.0,101.0,41.0
1,0,64,163.0,60.0,8.0,85.0,39.7
2,0,51,161.0,64.0,7.0,84.0,39.8
3,1,20,192.0,90.0,25.0,105.0,40.7
4,0,38,166.0,61.0,25.0,102.0,40.6


## detect outliers used 3 not 1.5 to make the iqr less strict 

In [6]:

# Check outliers for numeric columns only
numeric_cols = x.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    Q1 = x[col].quantile(0.25)
    Q3 = x[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    
    outliers = x[(x[col] < lower_bound) | (x[col] > upper_bound)]
    
    print(f"{col}: {len(outliers)} outliers")
    



Sex: 0 outliers
Age: 0 outliers
Height: 0 outliers
Weight: 0 outliers
Duration: 0 outliers
Heart_Rate: 0 outliers
Body_Temp: 0 outliers


## split Data to train , validate , test 

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42) 

## Scaling data 

In [8]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)


## Gride search to pick best paramter and model 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# RMSE scorer
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Define models and their parameter grids
models_params = [
    {
        "model": [ElasticNet(max_iter=50000)],
        "alpha": [0.001, 0.01, 0.1, 1],
        "l1_ratio": [0.1, 0.5, 0.9]
    },
    {
        "model": [Ridge(max_iter=50000)],
        "alpha": [0.01, 0.1, 1, 10]
    },
    {
        "model": [Lasso(max_iter=50000)],
        "alpha": [0.001, 0.01, 0.1, 1]
    }
]

best_score = np.inf
best_model = None
best_params = None

for mp in models_params:
    model = mp["model"][0]  # current model
    params = {k: v for k, v in mp.items() if k != "model"}  # hyperparameters only
    
    grid = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring=rmse_scorer,
        # cross validation to k = 3 because data is large
        cv=3,
        # to use all cpu cores
        n_jobs=-1
    )
    grid.fit(x_train, y_train)
    
    # Update best model if current RMSE is lower
    if -grid.best_score_ < best_score:  # multiply by -1 because scorer is negative
        best_score = -grid.best_score_
        best_model = grid.best_estimator_
        best_params = grid.best_params_

print("Best Model:", best_model)
print("Best Parameters:", best_params)
print("Best RMSE:", best_score)

# Evaluate on validation set
y_val_pred = best_model.predict(x_val)
val_rmse = rmse(y_val, y_val_pred)
print("Validation RMSE:", val_rmse)


Best Model: Ridge(alpha=0.1, max_iter=50000)
Best Parameters: {'alpha': 0.1}
Best RMSE: 11.110346417822367
Validation RMSE: 11.094066207833228


## Read test Data and applay same preprossecing on it 

In [10]:
df_test= pd.read_csv(r"D:\Downloads\MIA\Task3\f-1-racer-diet-planning\test.csv")

x_test = df_test.drop(columns=['id'])
x_test['Sex'] = np.where(x_test['Sex'] == 'male', 1, 0)
x_test_scaled = scaler.transform(x_test)

## preduction submission 

In [12]:
y_test_pred = grid.predict(x_test_scaled)

submission = pd.DataFrame({
    "id": df_test["id"],
    "Calories": np.round(y_test_pred, 3)
})
submission.to_csv("submissionTsk3.2model1.csv", index=False)